In [1]:
import glob
import itertools
import numpy as np
import os
import pandas as pd
import pybedtools as pbt
import statsmodels
import subprocess
import sys
import time

sys.path.append("/dors/capra_lab/users/fongsl/tools/py_/")
import config_readwrite as crw
import fet
import split_filename

sys.path.append("/dors/capra_lab/users/fongsl/tools/genome/")
import chr_functions

In [2]:
name = "/data/hodges_lab/ATAC-STARR_B-cells/bin_human-evolution/config"
config, configfile_name = crw.read_config(name)

BKGD = config["BKGD"]["merged_sharedacc_0000"]  # background to shuffle in
REGIONS_PATH = config["CIS_TRANS"]["regions_dir"]

RE = config["EQTL"]["results"]
FC_PY = config['VAR_ENRICHMENT']["bin"]  # script to calculate fold-change
FC_SLURM = config['VAR_ENRICHMENT']["bin_slurm"]  # script to calculate fold-change

# functions to intersect bed files 

In [3]:
def calculate_fc(py, bed, gwas, bkgd, outfile):
    cmd = f'python {py} {bed} {gwas} {bkgd} -o {outfile} -i 1000 -n 1'
    print("\n\nrunning", bed)
    print(cmd)
    subprocess.call(cmd, shell = True)

In [4]:
def calculate_slurm(slurm, bed, gwas, bkgd, outfile, nthreads):
    cmd = f'sbatch {slurm} {bed} {gwas} {bkgd} {outfile} {nthreads}'
    print("\n\nrunning slurm", bed)
    subprocess.call(cmd, shell = True)

# launch eQTL enrichment

In [5]:
tissues = ["ebv", "whole_blood", "spleen"]

## rerun

In [ ]:
tissue_act = itertools.product(tissues, test_filenames)
rerun =[]
for t, tt in tissue_act:
    print(t, tt)
    if (t == "spleen") & tt =="MM-active_HH-inactive_trans-only.bed":
        rerun.append((t,tt))

In [ ]:
rerun

In [ ]:
for t, f in rerun:
    print(t, f)

In [6]:
## file names to test for enrichment:

test_filenames = [
                "cis_only.bed", # spleen, ebv, whole_blood
                "cis+trans.bed", # spleen, ebv, whole_blood
                "trans_only.bed", # spleen only
                "conserved_active.regions.bed", # spleen only
    
                "HH-active_MM-inactive_cis-only.bed", # spleen only
                "HH-active_MM-inactive_cis+trans.bed", # spleen only
                "HH-active_MM-inactive_trans-only.bed", # spleen only
    
                "MM-active_HH-inactive_cis-only.bed", # spleen only
                "MM-active_HH-inactive_cis+trans.bed", # spleen only
                "MM-active_HH-inactive_trans-only.bed" # spleen only
                ]

In [7]:
val = 0
tissue_act = itertools.product(tissues, test_filenames)
for t, f in tissue_act:
#for t, f in rerun:
    
    full_bed = os.path.join(REGIONS_PATH, f)  # bedfile to test
    
    #calculate_fc(FC_PY, full_bed, GWAS, BKGD, OUTFILE)
    
    N_THREADS = 10

    OUTFILE = config["EQTL"][f"results_{t}"]
    EQTL = config["EQTL"][t]

    #calculate_fc(FC_PY, full_bed, EQTL, BKGD, OUTFILE)
    
    calculate_slurm(FC_SLURM, full_bed, EQTL, BKGD, OUTFILE, N_THREADS)

    val +=1




running slurm /data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/regions/cis_only.bed


running slurm /data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/regions/cis+trans.bed


running slurm /data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/regions/trans_only.bed


running slurm /data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/regions/conserved_active.regions.bed


running slurm /data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/regions/HH-active_MM-inactive_cis-only.bed


running slurm /data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/regions/HH-active_MM-inactive_cis+trans.bed


running slurm /data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/regions/HH-active_MM-inactive_trans-only.bed


running slurm /data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/regions/MM-active_HH-inactive_cis-only.bed


running slurm /data/hodges_lab/ATAC-STARR_B-cell

In [ ]:
f, t = "MM-active_HH-inactive_trans-only.bed", "spleen"
full_bed = os.path.join(REGIONS_PATH, f)  # bedfile to test

#calculate_fc(FC_PY, full_bed, GWAS, BKGD, OUTFILE)

N_THREADS = 10

OUTFILE = config["EQTL"][f"results_{t}"]
EQTL = config["EQTL"][t]

#calculate_fc(FC_PY, full_bed, EQTL, BKGD, OUTFILE)

calculate_slurm(FC_SLURM, full_bed, EQTL, BKGD, OUTFILE, N_THREADS)